In [ ]:
import numpy as np

## Online algorithm for the mean

Univariate

In [ ]:
def update_mean(prev_mean,new_data,n):
    new_mean = prev_mean + 1/float(n)*(new_data-prev_mean)
    return new_mean

In [ ]:
print update_mean(0,1,1)
print update_mean(1,0,2)


In [ ]:
values = np.random.randint(0,9,size=(10000))

In [ ]:
values


In [ ]:
values.mean()

In [ ]:
prev_mean = values[0]
n = 1
for new_data in values[1:]:
    n+=1
    new_mean = update_mean(prev_mean,new_data,n)
    prev_mean = new_mean
    print new_mean 
    

In [ ]:
def rolling_expand(arr):
    assert len(arr.shape) == 1, "Array must be 1-dimensional"
    padded_array = np.concatenate([np.nan*np.empty(shape=arr.shape[0]-1),arr],axis=0)
    return np.lib.stride_tricks.as_strided(padded_array,shape=(arr.shape[0],arr.shape[0]),strides=(8,8),writeable=False)
    

In [ ]:
rolling_means = np.nanmean(rolling_expand(values),axis=0)

In [ ]:
rolling_means.shape

In [ ]:
rolling_means[-1]

In [ ]:
values.shape

In [ ]:
values.strides

In [ ]:
values.shape = (10,10)

In [ ]:
values = values.reshape(10)

In [ ]:
values.strides

In [ ]:
values

In [ ]:
np.lib.stride_tricks.as_strided?

In [ ]:
np.lib.stride_tricks.as_strided(values,shape=(3,8),strides=(8,8),writeable=False)

In [ ]:
values

In [ ]:
rvalues = np.flip(values,0)

In [ ]:
rvalues.strides

In [ ]:
dvalues = np.concatenate([np.zeros(shape=(len(values)-1))*np.nan,values],axis=0)

In [ ]:
dvalues.shape

In [ ]:
rolled = np.lib.stride_tricks.as_strided(dvalues,shape=(10,10),strides=(8,8),writeable=False)

In [ ]:
rolled.nanmean(axis=1)

In [ ]:
np.nanmean(rolled,axis=1)

## Online covariance

In [ ]:
def update_cov_matrix(cov,xn,xn_mean,xn1_mean):
    return cov + np.matmul((xn-xn_mean)[None,:].transpose(),(xn-xn1_mean)[None,:])

In [ ]:
real_cov = np.array([[5,0.5],[0.5,1]])
real_mean = np.array([0,10])

In [ ]:
real_cov, real_mean

In [ ]:
N = 1000

In [ ]:
x = np.random.multivariate_normal(real_mean,real_cov,size=N)
estimated_s = 0
prev_mean = x[0]
for i, xi in enumerate(x[1:]):
    new_mean = update_mean(prev_mean,xi,i+1)
    estimated_s = update_cov_matrix(estimated_s,xi,new_mean,prev_mean)
    prev_mean = new_mean
    
estimated_s/float(N)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(x[:,0],x[:,1])

##  Online covariance in batches

## Draft for outlier detector algo

In [ ]:
class MockSelf(object):
    pass
self = MockSelf()

In [ ]:
# Ground truth

real_cov = np.array([[5,0.5],[0.5,1]])
real_mean = np.array([0,10])

In [ ]:
# state
self.mean = None
self.n = 0
self.C = 0

In [ ]:
# inputs
features = np.random.multivariate_normal(real_mean,real_cov,size=1)
feature_names = []

# score function

self.n += 1
feat0 = features[0]

if self.mean is None:
    self.mean = feat0
    assert False, 0
else:
    new_mean = self.mean + 1./float(self.n)*(feat0 - self.mean)
    self.C = self.C + np.matmul((feat0 - self.mean)[None,:].transpose(),(feat0 - new_mean)[None,:])
    self.mean = new_mean
    
mean = self.mean
cov = self.C/float(self.n-1)

cov1 = np.linalg.inv(cov)

# Mahalanobis distance = (X-u)t * C-1 * (X-u)
d2 = np.matmul(feat0 - mean, np.matmul(feat0 - mean, cov1.transpose()).transpose())

print features, d2

## Handling Batches

In [ ]:
self.n = 0 
self.mean = 0
self.C = 0

if $A$ and $A+B$ are invertible and $rank(B) = 1$ then

$ (A + B)^{-1} = A^{-1} - \frac{1}{1+trace(BA^{-1})}A^{-1}BA^{-1} $

In our case rank(B) is 1 because B is built from a single vector, so this applies.

In [ ]:
# inputs
features = np.random.multivariate_normal(real_mean,real_cov,size=1000)
feature_names = []

nb = features.shape[0]

roll_partial_means = features.cumsum(axis=0)/(np.arange(nb)+1).reshape((nb,1))

coefs = (np.arange(nb)+1.)/(np.arange(nb)+self.n+1.)

new_means = self.mean + coefs.reshape((nb,1))*(roll_partial_means-self.mean)

# lets compute rolling (Xn-mXn)t*(Xn-mXn-1)

new_means_offset = np.empty_like(new_means)
new_means_offset[0] = self.mean
new_means_offset[1:] = new_means[0:-1]

B = np.matmul((features - new_means)[:,:,None],(features - new_means_offset)[:,None,:])

all_C = self.C + B.cumsum(axis=0)
all_C_inv = np.zeros_like(B)

all_C = np.roll(all_C,1)
all_C[0] = self.C

c_inv = None

for i, b in enumerate(B):
    if c_inv is None:
        try:
            c_inv = np.linalg.inv(all_C[i])
        except np.linalg.LinAlgError:
            continue
    else:
        c_inv = all_C_inv[i-1]
    BC1 = np.matmul(b,c_inv)
    all_C_inv[i] = c_inv - 1./(1.+np.trace(BC1))*np.matmul(c_inv,BC1)
    

self.C += B.sum(axis=0)

self.mean = new_means[-1]

self.n += nb

In [ ]:
np.roll(np.arange(10),1)

In [ ]:
all_C_inv[:10]

In [ ]:
all_C[:10]

In [ ]:
try:
    np.linalg.inv(self.C)
except np.linalg.LinAlgError:
    print 'yo'

In [ ]:
C1 = np.linalg.inv(self.C)

In [ ]:
all_C = self.C + B.cumsum(axis=0)

In [ ]:
C1BC1 = np.matmul(C1,np.matmul(B,C1))

In [ ]:
C1BC1.shape

In [ ]:
all_C_inv.shape

In [ ]:
i = 980
np.abs(all_C_inv[i] - np.linalg.inv(all_C[i])).max().round(8)

In [ ]:
(np.arange(5)*0.23352346256).round(4)

In [ ]:
C1

In [ ]:
all_C_inv[1]

In [ ]:
np.linalg.inv(all_C[1])

In [ ]:
np.roll(np.arange(10),1)

In [ ]:
all_C.shape

In [ ]:
self.C

In [ ]:
self.C/(self.n-1.)

In [ ]:
self.n

In [ ]:
self.n

In [ ]:
features.shape

In [ ]:
np.multiply(features,features.transpose()).shape

In [ ]:
np.matmul(features[0:1].transpose(),features[0:1])

In [ ]:
features[:,None,:].shape

In [ ]:
np.matmul?

In [ ]:
features[0:1]

In [ ]:
B = 

In [ ]:
partial_means = 

In [ ]:
np.roll?

In [ ]:
features

In [ ]:
roll_partial_means = features.cumsum(axis=0)/(np.arange(nb)+1).reshape((3,1))

In [ ]:
cov

In [ ]:
cov

In [ ]:
cov1

In [ ]:
d2

In [ ]:
L = np.linalg.cholesky(cov)

In [ ]:
L1 = np.linalg.inv(L)

In [ ]:
mean

In [ ]:
np.matmul(L1,features[0]-mean)

In [ ]:
features = np.random.multivariate_normal(real_mean,real_cov,size=1000)

In [ ]:
features-mean

In [ ]:
cho_features = np.matmul(features-mean,L1.transpose())

In [ ]:
x_wit = np.random.multivariate_normal([0,0],np.diag(np.ones(2)),size=1000)

In [ ]:
cho_features[0]

In [ ]:
np.matmul(feat0-mean,L1.transpose())

In [ ]:
np.square(cho_features).sum(axis=1)

In [ ]:
np.sq

In [ ]:
d2 = np.square(cho_features).sum(axis=1)

In [ ]:
fig = plt.figure(figsize=(15,8))
plt.scatter(cho_features[:,0],cho_features[:,1],c = np.square(cho_features).sum(axis=1))
#plt.scatter(x_wit[:,0],x_wit[:,1])

In [ ]:
cov1 = np.linalg.inv(cov)

In [ ]:
d2[0]

In [ ]:
d2_2 = (np.matmul(cov1,(features-mean).transpose()).transpose()*(features-mean)).sum(axis=1)

In [ ]:
np.matmul(feat0 - mean, np.matmul(feat0 - mean, cov1.transpose()).transpose())

In [ ]:
fig = plt.figure(figsize=(15,8))
plt.scatter(features[:,0],features[:,1],c = d2_2)

In [ ]:
cov1.shape

In [ ]:
(features-mean).shape

In [ ]:
np.matmul?

In [ ]:
L1

In [ ]:
feat0[None,:]-mean[None,:]

In [ ]:
print cov
print mean

In [ ]:
features